In [1]:
import pdb

import requests
import pandas as pd
import re

In [2]:
def has_key(key, resp, pageid):
    return bool(resp['query']["pages"][pageid].get(key))

def matches(lst1, lst2):
    return [x for x in lst1 if x in lst2]
    
    
def get_article_info(title):
    params = {
        "action": "query",
        "format": "json",

        "titles": title,

        "prop": "info|links|linkshere|categories",
        
        # links
        "pllimit": "max",
        "plnamespace": 0,
        
        # linkshere
        "lhlimit": "max",
        "lhnamespace": 0,
        "lhshow": "!redirect",
        
        # categories
        "cllimit": "max",
    }

    links = []
    linkshere = []
    categories = []

    def query_info(title, params):

        resp = requests.get(
            url="https://en.wikipedia.org/w/api.php",
            params=params).json()

        pageid = list(resp["query"]['pages'].keys())[0]

        if has_key("links", resp, pageid):
            for link in resp['query']["pages"][pageid]["links"]:
                links.append(link["title"])
        
        if has_key("linkshere", resp, pageid):
            for lh in resp['query']["pages"][pageid]["linkshere"]:
                linkshere.append(lh["title"])
        
        if has_key("categories", resp, pageid):
            for cat in resp['query']["pages"][pageid]["categories"]:
                if not bool(re.findall(r"(articles)|(uses)|(commons)", cat["title"], re.I)):
                    categories.append(cat["title"])
        
        if resp.get('continue'):
            params.update(resp.get("continue"))
            query_info(title, params)

    query_info(title, params)
    
    return links, linkshere, categories

In [24]:
a, b, c = get_article_info("L%27Hôpital%27s_rule")
c

[]

In [3]:
links1, linkshere1, categories1 = get_article_info("Random forest")

links2, linkshere2, categories2 = get_article_info("Decision tree")

In [4]:
matches(categories1, categories2)

['Category:Decision trees']

In [5]:
related_articles = pd.read_csv("../../data/raw/expanded_labeled_filtered.zip")

In [17]:
test_set = related_articles[0:50].copy()

In [21]:
for i, row in test_set.iterrows():
    
    parent_links, parent_linkshere, parent_categories = get_article_info(row["parent_title"])
    child_links, child_linkshere, child_categories = get_article_info(row["child_title"])
    print("Parent:", row["parent_title"], len(parent_links), row["parent_url"])
    print("Child:", row["child_title"], len(child_links), row["child_url"])
    print("Combined:", len(matches(parent_links, child_links)))
    print("\n\n")

Parent: Division by Zero (story) 15 https://en.wikipedia.org/wiki/Division_by_Zero_(story)
Child: Division_by_zero 110 https://en.wikipedia.org/wiki/Division_by_zero
Combined: 2



Parent: Division by Zero (story) 15 https://en.wikipedia.org/wiki/Division_by_Zero_(story)
Child: Infinity 284 https://en.wikipedia.org/wiki/Infinity
Combined: 1



Parent: Division by Zero (story) 15 https://en.wikipedia.org/wiki/Division_by_Zero_(story)
Child: Zero 3 https://en.wikipedia.org/wiki/Zero
Combined: 0



Parent: Indeterminate form 26 https://en.wikipedia.org/wiki/Indeterminate_form
Child: Defined_and_undefined 1 https://en.wikipedia.org/wiki/Defined_and_undefined
Combined: 0



Parent: Indeterminate form 26 https://en.wikipedia.org/wiki/Indeterminate_form
Child: Division_by_zero 110 https://en.wikipedia.org/wiki/Division_by_zero
Combined: 10



Parent: Indeterminate form 26 https://en.wikipedia.org/wiki/Indeterminate_form
Child: Extended_real_number_line 90 https://en.wikipedia.org/wiki/Extende

Parent: Foundations of mathematics 376 https://en.wikipedia.org/wiki/Foundations_of_mathematics
Child: Liar_paradox 195 https://en.wikipedia.org/wiki/Liar_paradox
Combined: 15



Parent: Foundations of mathematics 376 https://en.wikipedia.org/wiki/Foundations_of_mathematics
Child: New_Foundations 152 https://en.wikipedia.org/wiki/New_Foundations
Combined: 56



